In [1]:
import os
from osgeo import gdal
import numpy as np
    
band_path = "../../data/images"
band_numbers = ['2', '3', '4', '5', '6', '7', '8', '8A', '11', '12']
band_date = [
    "_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220405-105855-542_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220714-105903-040_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220922-105859-863_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20221111-105858-090_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20230219-105857-687_L2A_T31TCJ_C_V3-1_FRE_B"
]  # Liste des dates pour chaque image
    
# Générer les chemins complets pour chaque bande
band_files = [f"{band_path}/SENTINEL2B{date}{band_number}.tif" for date in band_date for band_number in band_numbers]

shp_path = "../../data/project/emprise_etude.shp"
    
# Répertoire de sortie
out_path = "../results/data/img_pretraitees"
mask_file = os.path.join(out_path, "masque_foret.tif")  # Fichier de masque
    
# Paramètres de sortie
output_file = os.path.join(out_path, "Serie_temp_S2_allbands.tif")
output_ndvi_temp = os.path.join(out_path, "Serie_temp_S2_ndvi_temp.tif")
output_ndvi = os.path.join(out_path, "Serie_temp_S2_ndvi.tif")
projection = "EPSG:2154"
pixel_size = 10  # Résolution spatiale en mètres
nodata_value = 0

In [2]:
# Étape 1 : Fonction pour reprojeter et rééchantillonner une bande
def reproject_and_resample(band_file, output_file, pixel_size, projection, nodata_value):
    warp_command = (
        f"gdalwarp -tr {pixel_size} {pixel_size} -t_srs {projection} {band_file} {output_file}"
    )
    os.system(warp_command)

# Liste pour stocker les bandes reprojetées
reprojected_bands = []

# Boucle sur les dates et bandes pour reprojeter et rééchantillonner
for date_idx, date in enumerate(band_date, start=1):  # Itérer sur chaque date
    for band_idx, band_number in enumerate(band_numbers, start=1):  # Itérer sur chaque bande
        band_file = f"{band_path}/SENTINEL2B{date}{band_number}.tif"  # Générer le chemin complet
        reprojected_file = os.path.join(out_path, f"date_{date_idx}_band_{band_number}_reprojected.tif")  # Fichier de sortie
        reproject_and_resample(band_file, reprojected_file, pixel_size, projection, nodata_value)  # Reprojection
        reprojected_bands.append(reprojected_file) 

Creating output file that is 10996P x 10995L.
Processing ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10996P x 10995L.
Processing ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B3.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10996P x 10995L.
Processing ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B4.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10996P x 10995L.
Processing ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B5.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10996P x 10995L.
Processing ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B6.tif [1/1] : 0...10...20...30...40...50...60...70...80

In [3]:
# Étape 2 : Découper chaque bande à l'emprise du Shapefile
def clip_to_shapefile(input_file, output_file, shapefile_path, nodata_value):
    warp_command = (
        f"gdalwarp -cutline {shapefile_path} -crop_to_cutline {input_file} {output_file}"
    )
    os.system(warp_command)

# Appliquer le découpage à l'emprise pour chaque bande
clipped_files = []
for i, band_file in enumerate(reprojected_bands):
    clipped_file = os.path.join(out_path, f"band_{i+1}_clipped.tif")
    clip_to_shapefile(band_file, clipped_file, shp_path, nodata_value)
    clipped_files.append(clipped_file)

Creating output file that is 10862P x 7380L.
Processing ../results/data/img_pretraitees/date_1_band_2_reprojected.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Processing ../results/data/img_pretraitees/date_1_band_3_reprojected.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Processing ../results/data/img_pretraitees/date_1_band_4_reprojected.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Processing ../results/data/img_pretraitees/date_1_band_5_reprojected.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Processing ../results/data/img_pretraitees/date_1_band_6_reprojected.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Processing ../result

In [4]:
# Étape 3 : Appliquer le masque forêt à chaque bande
masked_bands = []
for i, band_file in enumerate(clipped_files):
    masked_file = os.path.join(out_path, f"band_{i+1}_masked.tif")
    gdal_calc_command = (
        f"gdal_calc.py -A {band_file} -B {mask_file} --outfile={masked_file} --calc=A*B --NoDataValue={nodata_value}"
    )
    os.system(gdal_calc_command)
    masked_bands.append(masked_file)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...

In [5]:
# Étape 4 : Fusionner toutes les bandes dans un seul fichier multidimensionnel
merge_command = (
    f"gdal_merge.py -separate -o {output_file} " + " ".join(masked_bands) + f" -a_nodata {nodata_value} -ot UInt16"
)
os.system(merge_command)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [6]:
# Étape 5 : Calculer le NDVI à partir des bandes Rouge (B4) et NIR (B8)

# Nouvelle valeur NoData
nodata_value = -9999

def calculate_ndvi(red_band, nir_band, output_file, nodata_value):

    # Ouvrir les images pour les bandes Rouge et NIR
    red_ds = gdal.Open(red_band)
    nir_ds = gdal.Open(nir_band)

    red_band_data = red_ds.ReadAsArray().astype(np.float32)
    nir_band_data = nir_ds.ReadAsArray().astype(np.float32)

    red_band_data = np.nan_to_num(red_band_data, nan=nodata_value, posinf=nodata_value, neginf=nodata_value)
    nir_band_data = np.nan_to_num(nir_band_data, nan=nodata_value, posinf=nodata_value, neginf=nodata_value)

    # Calcul du NDVI
    sum_bands = nir_band_data + red_band_data

    # Calculer NDVI en évitant la division par zéro
    ndvi = np.where(sum_bands != 0, (nir_band_data - red_band_data) / sum_bands, nodata_value)

    # Sauvegarder le NDVI dans un fichier
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output_file, red_ds.RasterXSize, red_ds.RasterYSize, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(red_ds.GetGeoTransform())
    out_ds.SetProjection(red_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(ndvi)
    out_band.SetNoDataValue(nodata_value)
    out_ds = None  # Fermer le fichier

# Liste des fichiers des bandes Rouge (B4) et NIR (B8) pour chaque date
red_band_files = [os.path.join(out_path, f"date_{date_idx}_band_4_reprojected.tif") for date_idx in range(1, len(band_date)+1)]
nir_band_files = [os.path.join(out_path, f"date_{date_idx}_band_8_reprojected.tif") for date_idx in range(1, len(band_date)+1)]

# Liste pour stocker les fichiers NDVI
ndvi_files = []

# Calculer le NDVI pour chaque paire de bandes Rouge et NIR
for date_idx in range(len(band_date)):
    output_ndvi_file = os.path.join(out_path, f"ndvi_date_{date_idx+1}.tif")
    calculate_ndvi(red_band_files[date_idx], nir_band_files[date_idx], output_ndvi_file, nodata_value)
    ndvi_files.append(output_ndvi_file)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/tmp/ipykernel_718967/2030819404.py:22: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where(sum_bands != 0, (nir_band_data - red_band_data) / sum_bands, nodata_value)


In [7]:
# Étape 6 : Découper chaque image NDVI à l'emprise du shapefile
def clip_to_shapefile(input_file, output_file, shapefile_path, nodata_value):
    warp_command = (
        f"gdalwarp -cutline {shapefile_path} -crop_to_cutline -dstnodata {nodata_value} -ot Float32 {input_file} {output_file}"
    )
    os.system(warp_command)

# Appliquer la découpe à chaque fichier NDVI séparément
clipped_ndvi_files = []
for idx, ndvi_file in enumerate(ndvi_files):
    clipped_file = os.path.join(out_path, f"ndvi_date_{idx+1}_clipped.tif")
    clip_to_shapefile(ndvi_file, clipped_file, shp_path, nodata_value)
    clipped_ndvi_files.append(clipped_file)

Creating output file that is 10862P x 7380L.
Using internal nodata values (e.g. -9999) for image ../results/data/img_pretraitees/ndvi_date_1.tif.
Processing ../results/data/img_pretraitees/ndvi_date_1.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Using internal nodata values (e.g. -9999) for image ../results/data/img_pretraitees/ndvi_date_2.tif.
Processing ../results/data/img_pretraitees/ndvi_date_2.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Using internal nodata values (e.g. -9999) for image ../results/data/img_pretraitees/ndvi_date_3.tif.
Processing ../results/data/img_pretraitees/ndvi_date_3.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 10862P x 7380L.
Using internal nodata values (e.g. -9999) for image ../results/data/img_pretraitees/ndvi_date_4.tif.
Processing ../results/data/img_p

In [8]:
# Étape 7 : Fusionner toutes les images NDVI pour créer la série temporelle
merge_ndvi_command = (
    f"gdal_merge.py -separate -o {output_ndvi_temp} " + " ".join(clipped_ndvi_files) + f" -a_nodata {nodata_value} -ot Float32"
)
os.system(merge_ndvi_command)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [9]:
# Étape 8 : Appliquer le masque forêt au NDVI (masquer les zones non-forêt)
ndvi_ds = gdal.Open(output_ndvi_temp)

# Création du fichier de sortie pour le NDVI masqué
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_ndvi, ndvi_ds.RasterXSize, ndvi_ds.RasterYSize, ndvi_ds.RasterCount, gdal.GDT_Float32)

# Copie des métadonnées
out_ds.SetGeoTransform(ndvi_ds.GetGeoTransform())
out_ds.SetProjection(ndvi_ds.GetProjection())

# Ouverture du masque
mask_ds = gdal.Open(mask_file)
mask_data = mask_ds.ReadAsArray()

# Application du masque à chaque bande de l'image fusionnée
for i in range(1, ndvi_ds.RasterCount + 1):
    ndvi_band = ndvi_ds.GetRasterBand(i)
    out_band = out_ds.GetRasterBand(i)

    # Lecture des données de la bande NDVI
    ndvi_data = ndvi_band.ReadAsArray()

    # Appliquer le masque
    masked_data = np.where(mask_data == 1, ndvi_data, nodata_value)

    # Écriture des données masquées dans la nouvelle bande
    out_band.WriteArray(masked_data)
    out_band.SetNoDataValue(nodata_value)

# Fermer les fichiers
out_ds = None
ndvi_ds = None
mask_ds = None

In [10]:
# Listes des fichiers intermédiaires à supprimer
intermediate_files = (
    reprojected_bands  # Bandes reprojetées
    + masked_bands  # Bandes masquées
    + ndvi_files  # Fichiers NDVI calculés
    + clipped_files
    + clipped_ndvi_files
    + [output_ndvi_temp]
    + clipped_ndvi_files
)

# Supprimer les fichiers intermédiaires
for file_path in intermediate_files:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Fichier supprimé : {file_path}")
    else:
        print(f"Fichier introuvable (non supprimé) : {file_path}")

Fichier supprimé : ../results/data/img_pretraitees/date_1_band_2_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_3_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_4_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_5_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_6_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_7_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_8_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_8A_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_11_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_1_band_12_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_2_band_2_reprojected.tif
Fichier supprimé : ../results/data/img_pretraitees/date_2_band_3_reprojected.tif
Fichier supprimé : ../res